# Physical Dynamics Neural Network
### Overview
Physical Dynamics Neural Network (PhyDNet) is a neural network architecture for spatiotemporal forecasting. It utilises traditional ConvLSTM cells along with a new Physical Dynamics Cell that models differential equations in latent variable space. This allows for the underlying physics of a problem to be modelled.

### Dependencies
PyTorch 1.5.0 with CUDA 10.1

In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from skimage.measure import compare_ssim as ssim
import random
import time

from PhyDNet.models import ConvLSTM, PhyCell, EncoderRNN
from PhyDNet.constrain_moments import K2M

from dataset_utils import GenericDataset, SpatiotemporalDataset

torch.cuda.empty_cache()

In [25]:
N_SAMPLES = 10000
N_CHANNELS = 1
IM_HEIGHT = 64
IM_WIDTH = 64
FORECAST_HORIZON = 10
LAGS = 10
MNIST = False

if MNIST:
    
    X = np.load("data/mnist_test_seq.npy").swapaxes(0,1)
    X = np.expand_dims(X, 2)
    trainDataset = GenericDataset(X[0:700], FORECAST_HORIZON, LAGS)
    validDataset = GenericDataset(X[700:], FORECAST_HORIZON, LAGS)
    
else:
    
    X = np.random.normal(0, 1, (N_SAMPLES, N_CHANNELS, IM_HEIGHT, IM_WIDTH)).astype(np.float32)
    trainDataset = SpatiotemporalDataset(X, FORECAST_HORIZON, LAGS)
    validDataset = SpatiotemporalDataset(X, FORECAST_HORIZON, LAGS)

In [27]:
device = "cpu" #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root = '/dds/work/PrevisionsPV/codes/video_prediction/SBNet-for-video-prediction/dataset'
batch_size = 1
n_epochs = 1001
print_every = 1
eval_every = 5

train_loader = torch.utils.data.DataLoader(dataset=trainDataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(dataset=validDataset, batch_size=1, shuffle=False, num_workers=0)

constraints = torch.zeros((49,7,7)).to(device)
ind = 0
for i in range(0,7):
    for j in range(0,7):
        constraints[ind,i,j] = 1
        ind +=1    

    
def train_on_batch(input_tensor, target_tensor, encoder, encoder_optimizer, criterion,teacher_forcing_ratio):                
    encoder_optimizer.zero_grad()
    # input_tensor : torch.Size([batch_size, input_length, 1, 64, 64])
    input_length  = input_tensor.size(1)
    target_length = target_tensor.size(1)
    loss = 0
    for ei in range(input_length-1): 
        encoder_output, encoder_hidden, output_image,_,_ = encoder(input_tensor[:,ei,:,:,:], (ei==0) )
        loss += criterion(output_image,input_tensor[:,ei+1,:,:,:])

    decoder_input = input_tensor[:,-1,:,:,:] # first decoder input = last image of input sequence
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False 
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, output_image,_,_ = encoder(decoder_input)
            target = target_tensor[:,di,:,:,:]
            loss += criterion(output_image,target)
            decoder_input = target 
         
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, output_image,_,_ = encoder(decoder_input)
            decoder_input = output_image
            target = target_tensor[:,di,:,:,:]
            loss += criterion(output_image, target)
 
    # Moment Regularisation  encoder.phycell.cell_list[0].F.conv1.weight # size (nb_filters,in_channels,7,7)
    k2m = K2M([7,7]).to(device)
    for b in range(0,encoder.phycell.cell_list[0].input_dim):
        filters = encoder.phycell.cell_list[0].F.conv1.weight[:,b,:,:] # (nb_filters,7,7)
        
        m = k2m(filters.double()) 
        m  = m.float()   
        loss += criterion(m, constraints) # constrains is a precomputed matrix   
    loss.backward()
    encoder_optimizer.step()
    return loss.item() / target_length


def trainIters(encoder, n_epochs, print_every,eval_every):
    start = time.time()
    train_losses = []
    best_mse = float('inf')

    encoder_optimizer = torch.optim.Adam(encoder.parameters(),lr=0.0001)
    scheduler_enc = ReduceLROnPlateau(encoder_optimizer, mode='min', patience=3,factor=0.5,verbose=True)
    criterion = nn.MSELoss()
    
    for epoch in range(0, n_epochs ):
        t0 = time.time()
        loss_epoch = 0
        teacher_forcing_ratio = np.maximum(0 , 1 - epoch * 0.01)
        
        for i, (idx, input_tensor, target_tensor) in enumerate(train_loader, 0):
            #input_batch =  torch.Size([8, 20, 1, 64, 64])
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)
            loss = train_on_batch(input_tensor, target_tensor, encoder, encoder_optimizer, criterion, teacher_forcing_ratio)                                   
            loss_epoch += loss
            
        if (i%10)==0 :
            print(i, '/', len(train_loader) , ' loss= ' , loss)
            
        train_losses.append(loss_epoch)        
        if (epoch+1) % print_every == 0:
            print('epoch ',epoch,  ' loss ',loss_epoch , ' epoch time ',time.time()-t0)
            
        if (epoch+1) % eval_every == 0:
            mse, mae,ssim = evaluate(encoder,test_loader) 
            scheduler_enc.step(mse)                              
    return train_losses


def evaluate(encoder,loader):
    total_mse, total_mae,total_ssim,total_bce = 0,0,0,0
    with torch.no_grad():
        for i, (idx, input_tensor, target_tensor) in enumerate(loader, 0):
            #input_batch = torch.Size([8, 20, 1, 64, 64])
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)

            input_length = input_tensor.size()[1]
            target_length = target_tensor.size()[1]

            for ei in range(input_length-1):
                encoder_output, encoder_hidden, _,_,_  = encoder(input_tensor[:,ei,:,:,:], (ei==0))

            decoder_input = input_tensor[:,-1,:,:,:] # first decoder input= last image of input sequence
            predictions = []

            for di in range(target_length):
                decoder_output, decoder_hidden, output_image,_,_ = encoder(decoder_input, False, False)
                decoder_input = output_image
                predictions.append(output_image.cpu())

            input = input_tensor.cpu().numpy()
            target = target_tensor.cpu().numpy()
            predictions =  np.stack(predictions) # for MM: (10, batch_size, 1, 64, 64)
            predictions = predictions.swapaxes(0,1)  # (batch_size,10, 1, 64, 64)

            mse_batch = np.mean((predictions-target)**2 , axis=(0,1,2)).sum()
            mae_batch = np.mean(np.abs(predictions-target) ,  axis=(0,1,2)).sum() 
            total_mse += mse_batch
            total_mae += mae_batch
            
            for a in range(0,target.shape[0]):
                for b in range(0,target.shape[1]):
                    total_ssim += ssim(target[a,b,0,], predictions[a,b,0,]) / (target.shape[0]*target.shape[1]) 
            
            cross_entropy = -target*np.log(predictions) - (1-target) * np.log(1-predictions)
            cross_entropy = cross_entropy.sum()
            cross_entropy = cross_entropy / (batch_size*target_length)
            total_bce +=  cross_entropy
     
    print('eval mse ', total_mse/len(loader),  ' eval mae ', total_mae/len(loader),' eval ssim ',total_ssim/len(loader), ' eval bce ', total_bce/len(loader))        
    return total_mse/len(loader),  total_mae/len(loader), total_ssim/len(loader)



print('BEGIN TRAIN')
phycell =  PhyCell(input_shape=(16,16), input_dim=64, F_hidden_dims=[49], n_layers=1, kernel_size=(7,7), device=device) 
convlstm =  ConvLSTM(input_shape=(16,16), input_dim=64, hidden_dims=[128,128,64], n_layers=3, kernel_size=(3,3), device=device)   
encoder = EncoderRNN(phycell, convlstm, device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
   
print('phycell ' , count_parameters(phycell) )    
print('convlstm ' , count_parameters(convlstm) ) 
print('encoder ' , count_parameters(encoder) ) 

plot_losses = trainIters(encoder,n_epochs,print_every=print_every,eval_every=eval_every)
print(plot_losses)

BEGIN TRAIN
layer  0 input dim  64  hidden dim  128
layer  1 input dim  128  hidden dim  128
layer  2 input dim  128  hidden dim  64
phycell  230833
convlstm  2508032
encoder  2951002


KeyboardInterrupt: 